# Introduction

In this notebook I try to make prediction on the Gold price using machine learning methods. The workflow of this prediction and its steps are as the following:
### Data Preparation
After importing the data which is the Gold price from August of 2000 to October of 2022, we need to prepare the dataset by feature engineering and exploratory data analysis. Briefly, we add lag columns and their statistics. At the end of this part, I defined a function that do all we need we some arguments.

### Machine Learning Model
In this section, I chose LGBMRegressor as the ML model for prediciting the price and tuned it with verstack.

### Trading Strategy
Finally, I created a trading strategy using the price we predicted in the last section which return the profit we make, if we trust to the model.

### Result
We know that financial markets are very "Complex" and need very high level mathematics like Stochastic Analysis for making any prediction or analysis.
Nevertheless, I did this type of prediction just for making sense of the market and I plan to use more mathematical approaches for this objective in the soon future and will publish it here.

### Profit
By the trading strategy mentioned above, we observe more than 20% profit in 3 month.

In [ ]:
!pip install verstack

In [ ]:
pip install git+https://github.com/OpenHydrology/lmoments3.git

In [ ]:
import statistics
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from scipy import stats
import lmoments3 as lm
from numpy import sqrt
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from lightgbm import LGBMRegressor
from verstack import LGBMTuner

In [ ]:
Gold = pd.read_csv(r'../input/gold-price-2000-2022/gold price - 2000 edited.csv',
                  header=0, index_col=0, parse_dates=True,squeeze=True)
Gold = Gold[Gold.Open !=0]
Gold = Gold[Gold.High !=0]
Gold = Gold[Gold.Low !=0]
Gold = Gold[Gold.Close !=0]
Gold

# Data Preparation
## Select best Lags

* The first function create the lags we specify and add them to the dataset.
* If we want to use lag for our time series forcasting, we should know which lag is better to add to our dataset.
* The second function use Mutual Information score to detect most relevant and important lags to the column we wish to predict or analysis.

In [ ]:
# add lags we set in "lag_list" and place them in order to the dataset.

def Add_Lag(data,col,lag_list):
    
    position = data.columns.tolist().index(col)
    
    for lag in lag_list:
    
        col_val = data[col].shift(lag)

        col_name = col+'_lag:'+'{length}'.format(length=lag)

        data.insert(loc=position, column=col_name, value=col_val)
        

    return data

In [ ]:
def mutual_information_lag(Data,col,n_lag,k_best):
    
    #create a dataset by selecting just one column from Gold dataset; a dataset with just one column of {Open,Close,High,Low,Volume} and time as index
    data = pd.DataFrame(Data[col])
    
    #create the dataset that its columns are lags with different shifts
    mic_df = Add_Lag(data,col,[i+1 for i in range(n_lag)])
    
    mic_ordered = {}
    
    #calculating the mutual information score between lag columns and target column; one of {Open,Close,High,Low,Volume} which selected above
    for i in range(1,n_lag+1):
        
        i_lag = mic_df[mic_df.columns[mic_df.shape[1]-i-1]][i:]
        i_lag_name = mic_df.columns[mic_df.shape[1]-i-1]
        score = mutual_info_regression(np.array(i_lag).reshape(-1, 1), mic_df[col][i:])
        mic_ordered[i_lag_name] = score[0]
    
    #descend sort the lag list by their mutual information score
    sorted_mic = {k: v for k, v in sorted(mic_ordered.items(),reverse=True, key=lambda item: item[1])}
    
    #select the k first lags that have most mutual information scor
    selected_lags = list(sorted_mic.keys())[:k_best]
    
    #create a list of best k lags obtained above
    select_lag_num = []
    for i in selected_lags:
        
        pos = i.find(':')
        select_lag_num.append(int(i[pos+1:]))
        
    select_lag_num_sort = sorted(select_lag_num)
    
    #draw a plot that visualize all lags (not just best k lags) and their mutul information score with the target
    plt.plot([i+1 for i in range(n_lag)], sorted_mic.values(),'--bo', label = "mutual information score",linestyle = 'dashed')
    plt.xticks(rotation = 90)njh
    plt.legend()
    plt.show()
    
    return select_lag_num_sort,sorted_mic,selected_lags,sorted_mic.values()

In [ ]:
#try the function on the "Close price" of the Gold data, calculating 100 first lags and select 10 high mutual information score lags
Best_Lags = mutual_information_lag(Gold,'Close',100,10)
#as we can see in the following almost best k lags are first k lags 

## Statistical Features
* In the following some summary statistics defined for adding to the dataset.
* For mathematical definitions of this function you can refer to the below notebook:
* https://www.kaggle.com/code/khashayarrahimi94/lgbm-using-summary-statistics-tps

In [ ]:
def gini_coefficient(x):
    #Compute Gini coefficient of array of values
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

In [ ]:
def Roll_Stats(data,col,Window):
    
    lag = 2
    
    """
    Important:
    
    Setting Lag=2 is because our trading strategy. By lag=2 these statistics computed from 2 days before prediction to more days back.
    
    """
    position = data.columns.tolist().index(col)

    col_val = data[col].shift(lag)
    window1 = col_val.rolling(window=Window)
    #window2 = col_val.rolling(window=Window+5)
    """
    Adding window by 3 because we need more general value for our min and max,
    Actually if we dont add 3, the min and max repeated because they existed in the data as lags
    """ 
    
    means = window1.mean()
    std = window1.var()
    #Max  = window2.max()
    #Min = window2.min()
    
    col_name_mean = col+'_mean'+'_lag:'+'{length}'.format(length=lag)+\
    '_win:'+'{length}'.format(length=Window)
    col_name_std = col+'_std'+'_lag:'+'{length}'.format(length=lag)+\
    '_win:'+'{length}'.format(length=Window)
    #col_name_max = col+'_max'+'_lag:'+'{length}'.format(length=lag)+\
    #'_win:'+'{length}'.format(length=Window)
    #col_name_min = col+'_min'+'_lag:'+'{length}'.format(length=lag)+\
    #'_win:'+'{length}'.format(length=Window)
    
    data.insert(loc=position, column=col_name_mean, value=means)
    data.insert(loc=position, column=col_name_std, value=std)
    #data.insert(loc=position, column=col_name_max, value=Max)
    #data.insert(loc=position, column=col_name_min, value=Min)
        

    return data

In [ ]:
def L_moment(data,col,Window):
    
    position = data.columns.tolist().index(col)
    
    Len = data.shape[0]-Window
    col_val = data[col].shift(1)
    
    col_name_1 = col+'_win:'+'{length}'.format(length=Window)+\
    '_L:'+'{moment}'.format(moment=1)
    col_name_2 = col+'_win:'+'{length}'.format(length=Window)+\
    '_L:'+'{moment}'.format(moment=2)
    col_name_3 = col+'_win:'+'{length}'.format(length=Window)+\
    '_L:'+'{moment}'.format(moment=3)
    col_name_4 = col+'_win:'+'{length}'.format(length=Window)+\
    '_L:'+'{moment}'.format(moment=4)
    col_name_5 = col+'_win:'+'{length}'.format(length=Window)+\
    '_L:'+'{moment}'.format(moment=5)
    col_name_6 = col+'_win:'+'{length}'.format(length=Window)+'_gini'
    
    L1 = [np.nan]*Window
    L2 = [np.nan]*Window
    L3 = [np.nan]*Window
    L4 = [np.nan]*Window
    L5 = [np.nan]*Window
    Gini = [np.nan]*Window
    
    for i in range(Len):
        
        lmoments = lm.lmom_ratios(col_val[i:i+Window].round(3).tolist(), nmom=5)
        gini = gini_coefficient(col_val[i:i+Window].round(3))
        
        L1.append(lmoments[0])
        L2.append(lmoments[1])
        L3.append(lmoments[2])
        L4.append(lmoments[3])
        L5.append(lmoments[4])
        Gini.append(gini)
        
    data.insert(loc=position, column=col_name_5, value=L5)
    data.insert(loc=position, column=col_name_4, value=L4)
    data.insert(loc=position, column=col_name_3, value=L3)
    data.insert(loc=position, column=col_name_2, value=L2)
    data.insert(loc=position, column=col_name_1, value=L1)
    data.insert(loc=position, column=col_name_6, value=Gini)
    
    return data

In [ ]:
# this is the final function that use above functions and return the prepared dataset
def Prepare_Data(data,col,Lag_list,Roll_window,Lmoment_window):
    
    if Roll_window <2:
        print('Roll_window must be greater than 1.')

    Add_Lag(data,col,Lag_list)
    Roll_Stats(data,col,Roll_window)
    L_moment(data,col,Lmoment_window)

    return data  

In [ ]:
#try the function
data = pd.DataFrame({'High':Gold['High']}, index = Gold.index)
data = Prepare_Data(data,'High',[2,3],2,5)
data

# Machine Learning Model

In [ ]:
def prediction(Data,col,Lag_list,Roll_window,Lmoment_window):

    data = pd.DataFrame({col:Data[col]}, index = Data.index)
    
    #prepare the target column we wish to predict
    data = Prepare_Data(data,col,Lag_list,Roll_window,Lmoment_window)
    
    #these three columns with their new features add to the final dataset, whatever the target column is. 
    Open = Prepare_Data(pd.DataFrame(Gold['Open']),'Open',Lag_list,Roll_window,Lmoment_window)
    Close = Prepare_Data(pd.DataFrame(Gold['Close']),'Close',Lag_list,Roll_window,Lmoment_window)
    Volume = Prepare_Data(pd.DataFrame(Gold['Volume']),'Volume',Lag_list,Roll_window,Lmoment_window)
    
    #Due to creating a complete dataset containg all features, if the target is "High" we add Low features and vice versa.
    if col == 'High':
        df = Prepare_Data(pd.DataFrame(Gold['Low']),'Low',Lag_list,Roll_window,Lmoment_window)
        
    elif col == 'Low':
        df = Prepare_Data(pd.DataFrame(Gold['High']),'High',Lag_list,Roll_window,Lmoment_window)
        
    #now we merge all the dataset we created above together to have a complete dataset 
    data = pd.concat([Open,Close,Volume,df,data],axis=1)
    
    #eliminate rows that have missing values due to using lags
    tail = data.shape[0]-max(Roll_window,len(Lag_list))
    data = data.tail(tail)
    
    #define train data
    X = data.values[:-90,:-1]
    Y = data.values[:-90,-1]

    #use lgbmtuner for hyperparaeter tuning
    tuner = LGBMTuner(metric = 'rmse',trials = 50) # <- the only required argument

    #the tuner needs these datatype for X and Y
    X = pd.DataFrame(X)
    Y = pd.Series(Y)
    
    #fit the tuned model and make prediction on the last 90 days of the data
    tuner.fit(X,Y)
    x_test = data.values[-90:,:-1]
    x_test = pd.DataFrame(x_test)
    predict = tuner.predict(x_test)
    
    y_test = pd.Series(data.values[-90:,-1])
    
    mae = mean_absolute_error(y_test, predict)
    rmse = sqrt(mean_squared_error(y_test, predict))
    
    
    """The best case for both diff in the following is small positive numbers
    which means the prediction is accurate and the trading strategy is possible.""" 
    
    diff_low = [] 
    if col=='Low':
        
        for i in range(len(predict)):
            
            diff = predict[i] - y_test[i]
        
            diff_low.append(diff)
            
        print ('Mean:',mean(diff_low),'Variance:',statistics.variance(diff_low))

            
    diff_high = [] 
    if col=='High':
        
        for i in range(len(predict)):
            
            diff =  y_test[i] - predict[i]
        
            diff_high.append(diff)
        
        print ('Mean:',mean(diff_high),'Variance:',statistics.variance(diff_high))
    
    
    
    print('MAE: %f' % mae)
    print('RMSE: %f' % rmse)
    print('======Summary======')
    print('Lag_list:','===>',Lag_list)
    print('Roll_window:','===>',Roll_window)
    print('Lmoment_window:','===>',Lmoment_window)
    #print('n_estimator:','===>',N_estimator)
    #print('Learing Rate:','===>',Learning_R)
    print('==============================')
    print('predict:\n',[ '%.1f' % y for y in predict ])
    print('Actual:\n', y_test.tolist())

    return predict.tolist(),y_test.tolist()

In [ ]:
low_prediction = prediction(Gold,'Low',[i for i in range(2,6)],2,5)

In [ ]:
x = data[-90:].index
x = pd.DataFrame(Gold.index[-90:]).values
# plot Low prediction and actual price
plt.plot(x, low_prediction[0],'--bo', label = "predict",linestyle = 'dashed')
plt.plot(x, low_prediction[1],marker='o', label = "Actual")
plt.xticks(rotation = 90)
plt.legend()
plt.show()

In [ ]:
high_prediction = prediction(Gold,'High',[i for i in range(2,6)],2,5)

In [ ]:
# plot High prediction and actual price
plt.plot(x, high_prediction[0],'--bo', label = "predict",linestyle = 'dashed')
plt.plot(x, high_prediction[1],marker='o', label = "Actual")
plt.xticks(rotation = 90)
plt.legend()
plt.show()

# Trading Strategy

Due to the limitation of information in our dataset, we can not trade daily based on "Low" and "High" price prediction.
The simplest strategy is buying at the Low price in a day and sell at the High price on that day. But since we dont have more detailed data including hourly price, we can not know the order of Low and High price; Which one occured before the other in each day?

Therefore, we create another strategy:

Since our prediction for day(i) is done by the information from day(j) where j < i-1 (means two days before day(i)), we have predictions for the Low price of day(i-1) and High price of day(i), we can buy in the low price of day(i-1) hope for High price of day(i) for selling. 

In [ ]:
def trading_strategy(money,pre_Low,pre_High,low_add,high_loss):
    
    """
    "low_add" and "high_loss" are two values we respectively add and subtract to and from low and high price predictions.
    By this, we raise the probability of trading and derease the profit, but if you try this approach and it's alternate
    (don't use these two values) you will convince to use them.
    """ 
    money_model = money
    money_optimal = money
    
    pre_low = pre_Low[0]
    pre_high = pre_High[0]
    
    act_low = pre_Low[1]
    act_high = pre_High[1]
    
    profit_model = 0
    profit_optimal = 0
    i = 0
    
    #By un-comment the following comments, you can see daily profit based on model prediction

    while i < 89:

            if (pre_low[i]+low_add >= act_low[i]) and (pre_low[i]+low_add < pre_high[i+1]):

                gold_bought = money_model/(pre_low[i]+low_add)
                #print('pre_low:',pre_low[i])
                #print('gold_bought:',i,gold_bought)
                
                i = i + 1
                j = i
                while j < 89:

                    if (act_high[j] >= pre_high[j]-high_loss):

                        diff_model = gold_bought * (pre_high[j]-high_loss) - money_model
                        #print('day {}: Diff'.format(j),diff_model)
                        profit_model = profit_model + diff_model
                        #print('day {}: Profit'.format(j),profit_model)
                        money_model = money_model + diff_model
                        #print('day {}: Money'.format(i),money_model)
                        #print('=============================================')
                        i = i + 1
                        break

                    else:
                        j = j + 1
                        i = j

            else:
                #print('last',i)

                i = i + 1
                
    final_profit_model = 0
    last_day_sell_pice = mean([pre_low[89], pre_high[89]])
    
    if last_day_sell_pice < act_high[89]:
                
        final_profit_model = max(profit_model,gold_bought*last_day_sell_pice)-1000
    
    # The below code snippet creat the "Ideal Case" which is when you know the exat low and high price and trade as above 
    
    j = 0
    while j < 89:

        if act_low[j] < act_high[j+1]:
            diff_optimal = ((money_optimal/act_low[j]) * act_high[j+1] - money_optimal)
            #print(diff_optimal)
            profit_optimal = profit_optimal + diff_optimal
            #print(profit_optimal)
            money_optimal = money_optimal + diff_optimal
            #print(money_optimal)

            j = j + 2
            #print(j)
            #print('*************')

        else:
            j = j + 1
            
    print('Optimal:')
    print('profit:',profit_optimal,'Capital:',money_optimal)
    print('============')
    print('Model:')
    print('profit:',final_profit_model,'Capital:',1000+final_profit_model)
    
    return final_profit_model,profit_optimal

In [ ]:
#Tuning the trading_strategy
def tune_strategy(low_prediction,high_prediction):
    profit_dict = {}
    for i in np.arange(0, 5, 0.1):

        for j in np.arange(0, 5, 0.1):

            profit = trading_strategy(1000,low_prediction,high_prediction,i,j)[0]
            
            profit_dict[(i,j)] = profit
    
    sorted_dict = {k: v for k, v in sorted(profit_dict.items(),reverse=True, key=lambda item: item[1])}
    
    best_parameters = list(sorted_dict.keys())[0]
    
    return best_parameters,sorted_dict


In [ ]:
best_param = tune_strategy(low_prediction,high_prediction)

In [ ]:
print('low_add:',best_param[0][0])
print('high_loss:',best_param[0][1])

In [ ]:
trading_strategy(1000,low_prediction,high_prediction,best_param[0][0],best_param[0][1])

### Without Parameter Tuning

In [ ]:
trading_strategy(1000,low_prediction,high_prediction,0,0)

# Result
**Where the maximum possible profit is <50% of the first fund which is 1000 dollor, our prediction help us to earn more than 20%.**